In [1]:
!pip install google -i http://pypi.douban.com/simple  --trusted-host pypi.douban.com

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install protobuf -i http://pypi.douban.com/simple  --trusted-host pypi.douban.com

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [50]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# convert tmall session data to our protobuf data

import google.protobuf
import google.protobuf.text_format
import chat_pb2
import json
import re

# line: 2018-05-31
def extract_date(line):
    # print line
    date_ptn = r'(\d{4}-\d{2}-\d{2})'
    m = re.match(date_ptn, line)
    if not m:
        return None
    date = m.group(1)
    return date

# line: a979289485 (11:43:39)
def extract_speaker(line):
    speaker_ptn = r'(.*)\((\d{2}:\d{2}:\d{2})\)'
    m = re.match(speaker_ptn, line)
    if not m:
        return None
    return m.group(1).strip()

# line: a979289485 (11:43:39)
def extract_time(line): 
    speaker_ptn = r'(.*)\((\d{2}:\d{2}:\d{2})\)'
    m = re.match(speaker_ptn, line)
    if not m:
        return None
    return m.group(2).strip()

# print extract_speaker('a979289485 (11:43:39)')

# convert taobao json to chat pb
def convert(inFile, outFile):
    print u'Convert ' + inFile + u' to ' + outFile
    taobao_chatbot_data_file = inFile

    with open(taobao_chatbot_data_file, 'r') as fin:
        json_data = json.load(fin)

    chat = chat_pb2.Chat()
    for session_data in json_data['tmall_kefu_data']:
        turn_data_index = 0
        date = ''
        session = chat.session.add()
        turn = None
        for turn_data in session_data['data']:
            
            # 第0个数，是发生日期
            # 第奇数个数，是speaker
            # 第偶数个数，是content
            content = turn_data['content']
            if not content:
                content = ''
            # print u'content ' + content
            if (u'暂无聊天记录' == content):
                continue
            extracted_date = extract_date(content)
            if extracted_date:  # each new date means we encounter a new turn
                date = extracted_date
                turn_data_index = 1
                continue
                
            if turn_data_index % 2 == 1:  # 奇数turn, extract speaker info
                turn = session.turn.add()
                speaker = extract_speaker(content)
                if not speaker:
                    print u'Failed to extract speaker from ' + content
                time = extract_time(content)
                if not time:
                    print u'Failed to extract time from ' + content
                turn.time = date + ' ' + time
                turn.speaker = speaker
            elif turn_data_index % 2 == 0:  # 偶数turn，extract content info
                query = ''
                if content:
                    query = content
                face_title = turn_data['face_title']
                face_url = turn_data['face_url']
                if face_title and face_url:
                    query = '[img_url=' + face_url + '][img_title =' + face_title + ']' + query
                turn.query.append(query)
            turn_data_index = turn_data_index + 1
        #print 'session:'
        #print session
        # print session.SerializeToString().encode('utf-8')
        # print text_format.MessageToString(session, as_utf8=True)
        # break
    # print 'Fin'
    #text_format.MessageToString(sessions[0], as_utf8=True)
    print u'Get session ' + str(len(chat.session))
    with open(outFile, "w") as fout:
        fout.write(chat.SerializeToString())
        
        
# convert( u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.json', u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb')


In [51]:
import glob
kf_json_list = glob.glob(u'/Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/*.json')
for kf_json in kf_json_list:
    convert(kf_json, kf_json + '.pb')


Convert /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs5_all_20180816.json to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs5_all_20180816.json.pb
Get session 15
Convert /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能表妹_all_20180816.json to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能表妹_all_20180816.json.pb
Get session 5253
Convert /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs4_all_20180816.json to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs4_all_20180816.json.pb
Get session 2
Convert /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs06_all_20180816.json to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs06_all_20180816.json.pb
Get session 6673
Convert /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs1_all_20180816.json to /Users/qli/Documents

In [53]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# annoymize the pb query which has user name, user address, and user phones

import google.protobuf
import chat_pb2
import re

cn_mobile = [134,135,136,137,138,139,150,151,152,157,158,159,182,183,184,187,188,147,178,1705]
cn_union = [130,131,132,155,156,185,186,145,176,1709]
cn_telecom = [133,153,180,181,189,177,1700]
def detect_cellphone(line):
    ptn1 = r'.*(\d{11}).*'
    m = re.match(ptn1, line)
    if not m:
        return False
    phone = m.group(1)
    ptn2 = '^(' + '|'.join([str(phone_prefix) for phone_prefix in cn_mobile + cn_union + cn_telecom]) + ')'
    if not re.match(ptn2, phone):
        return False
    return True

#print detect_cellphone('~~17633520709')

def anonymize_query(query):
    if detect_cellphone(query):
        # print u'anonymize query from ' + query + u' to <联系方式>.'
        return u'<联系方式>'
    else:
        return query
    
def anonymize(inFile, outFile):
    print u'Anonymize ' + inFile + ' to ' + outFile
    chat = chat_pb2.Chat()
    with open(inFile, 'r') as fin:
        chat.ParseFromString(fin.read())
    
    speakers = set()
    for session in chat.session:
        for turn in session.turn:
            anonymized_query = [anonymize_query(query) for query in turn.query]
            turn.ClearField('query')
            turn.query.extend(anonymized_query)
    
    with open(outFile, "w") as fout:
        fout.write(chat.SerializeToString())
        
# anonymize(u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb', \
#          u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb.anonymized')

In [63]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# anonymize a given dir.
import glob
kf_pb_list = glob.glob(u'/Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/*.json.pb')
for kf_pb in kf_pb_list:
    anonymize(kf_pb, kf_pb + '.anonymized')

Anonymize /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs09_all_20180816.json.pb to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs09_all_20180816.json.pb.anonymized
Anonymize /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能表姐_all_20180816.json.pb to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能表姐_all_20180816.json.pb.anonymized
Anonymize /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs06_all_20180816.json.pb to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs06_all_20180816.json.pb.anonymized
Anonymize /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs5_all_20180816.json.pb to /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs5_all_20180816.json.pb.anonymized
Anonymize /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能小表哥_all_20180816.json.pb to /Users/qli

In [66]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# Calculate statistics
import glob

class ChatStats:
    num_sessions = 0
    num_turns = 0
    
    def __init__(self, num_sessions, num_turns):
        self.num_sessions = num_sessions
        self.num_turns = num_turns
        
    def __str__(self):
        turns_per_session = self.num_turns / self.num_sessions
        return '\n'.join([\
            u'Total sessions ' + str(self.num_sessions), \
            u'Total turns ' + str(self.num_turns), \
            u'Turns per session ' + str(turns_per_session)\
            ]) 
        
def calculate_chat_statistics(inFile):
    print u'Calculate statistics ' + inFile
    chat = chat_pb2.Chat()
    with open(inFile, 'r') as fin:
        chat.ParseFromString(fin.read())
    
    num_sessions = len(chat.session)
    num_turns = 0
    for session in chat.session:
        num_turns = num_turns + len(session.turn)
        
    chat_stats = ChatStats(num_sessions, num_turns)
    print chat_stats
    return chat_stats

kf_pb_anonym_list = glob.glob(u'/Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/*.json.pb.anonymized')
chat_stats = [calculate_chat_statistics(kf_pb_anonym) for kf_pb_anonym in kf_pb_anonym_list]
total_sessions = 0
total_turns = 0
for c in chat_stats:
    total_sessions = total_sessions + c.num_sessions
    total_turns = total_turns + c.num_turns
print u'----------------------------------------------'
print u'Total sessions: ' + str(total_sessions)
print u'Total turns: ' + str(total_turns)
print u'Turns per session: ' + str(total_turns / total_sessions)

Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs3_all_20180816.json.pb.anonymized
Total sessions 12209
Total turns 197431
Turns per session 16
Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs6_all_20180816.json.pb.anonymized
Total sessions 3778
Total turns 57468
Turns per session 15
Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_智能表姐_all_20180816.json.pb.anonymized
Total sessions 7511
Total turns 144947
Turns per session 19
Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_cs11_all_20180816.json.pb.anonymized
Total sessions 11865
Total turns 140765
Turns per session 11
Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticwatch_tmall_all_20180816/ticwatch旗舰店_分销_all_20180816.json.pb.anonymized
Total sessions 194
Total turns 1805
Turns per session 9
Calculate statistics /Users/qli/Documents/墨百意/问问客服数据/ticw